# Generate hole region files
This is currently producing the simplest possible hole regions. For every star brighter than 16 Mag it puts a 10 arcsec circle. It then goes on to produce varying size holes for individual pristine catalogues based on parameters computed by Seb Oliver's IDL code.

In [1]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

This notebook was run with herschelhelp_internal version: 
79293bd (Wed Jun 21 16:47:54 2017 +0100) [with local modifications]


In [2]:
from herschelhelp_internal import starmask


First we list all the fields including the ra and dec or the first star in the field. We do this because, due to a peculiarity of the pyregion code, we must supply an image header to produce mocs.

In [3]:
#Field names plus ra dec of first circle
fields= [
# Field                RA              DEC
['AKARI-NEP',          274.654402036,  65.7962520276 ],
['AKARI-SEP',          72.2316923316, -54.380443672  ],
['Bootes',             216.431700722,  32.401081899  ],
['CDFS-SWIRE',         51.0227099923, -29.8185285737 ],
['COSMOS',             149.295925951,  1.08212668291 ],
['EGS',                217.276981956,  53.6441519854 ],
['ELAIS-N1',           247.096600963,  55.1757687739 ],
['ELAIS-N2',           248.424493154,  39.1274077489 ],
['ELAIS-S1',           7.10625839472, -43.8632559768 ],
['GAMA-09',            129.076050945, -2.23171513025 ],
['GAMA-12',            172.84437099,  -0.482115877707],
['GAMA-15',            211.756497623, -2.28573712848 ],
['HDF-N',              190.259734752,  62.205265532  ],
['Herschel-Stripe-82', 353.751913281, -7.10891111165 ],
['Lockman-SWIRE',      161.942787703,  59.0563805825 ],
['NGP',                192.899559129,  22.0990890388 ],
['SA13',               197.895801254,  42.4400105492 ],
['SGP',                334.297748942, -34.5037863499 ],
['SPIRE-NEP',          266.334305546,  68.7904496043 ],
['SSDF',               341.577544902, -59.1868365369 ],
['xFLS',               261.387059958,  58.0184602211 ],
['XMM-13hr',           203.318355937,  37.4745777866 ],
['XMM-LSS',            32.9413834032, -6.02293494708 ]]

Then we generate a region file to define all the holes. At present this is very crude and simply puts a 10 arcsec hole over every star brighter than 16 Mag.

In [ ]:
for field in fields:
    starmask.create_holes('../dmu0/dmu0_GAIA/data/GAIA_' + field[0] + '.fits',
                             'dmu4_sm_' + field[0] + '/data/10_arcsec_holes_' + field[0] + '.reg')
    


Then we convert the region files to MOC format. We recomend against using the MOC since, even at this high order, it doesn't capture shape that well. This will become increasingly significant if we have smaller or more detailed shapes in the future.

In [9]:
ORDER=16
for field in fields:
    starmask.reg2moc('dmu4_sm_' + field[0] + '/data/holes_' + field[0] + '.reg',
            '../dmu2/dmu2_field_coverages/' + field[0] + '_MOC.fits',
            'dmu4_sm_' + field[0] + '/data/10_arcsec_holes_' + field[0] + '_O' + str(ORDER) + '_MOC.fits',
            ra_typ=field[1],
            dec_typ=field[2],
            order=ORDER)


WCSAXES =                    2 / Number of coordinate axes
CRPIX1  =            [astropy.io.fits.card]
     0.5 / Pixel coordinate of reference point
CRPIX2  =                0.5 / Pi [astropy.io.fits.card]
xel coordinate of reference point
CDELT1  =                0.675 / [deg] Coordin [astropy.io.fits.card]
ate increment at reference point
CDELT2  =                0.675 / [deg] Coordina [astropy.io.fits.card]
te increment at reference point
RADECSYSa= 'ICRS    '           / International  [astropy.io.fits.card]
Celestial Ref. System
CUNIT1  = 'deg     '                / Units of coordinate  [astropy.io.fits.card]
increment and value
CUNIT2  = 'deg     '                / Units of coordinate in [astropy.io.fits.card]
crement and value
CTYPE1  = 'RA---TAN'           / 
CTYPE2  = 'DEC--TAN'         [astropy.io.fits.card]
   / 
CRVAL1  =                  247.0 / [deg] Coordinate value at reference poi [astropy.io.fits.card]
nt
CRVAL2  =                  55.1 / [deg] Coordinate value at r

FileNotFoundError: [Errno 2] No such file or directory: 'dmu4_sm_AKARI-NEP/data/holes_AKARI-NEP.reg'

## Produce magnitude varying holes
Then we produce a varying hole parameterised by Seb's code. We define an annulus with a 1 arcsec circle at the centre and an outer radius r_50 (the radius at which the artefact density goes over 0.5 x background density) = 10^(A + B x mag).

In [5]:
# For now just run ELAIS-N1 with Seb's emailed parameters
# TODO: Implement code to generate params and store them in dmu for all relevant prisitine catalogues/fields.
# Perhaps convert Seb's IDL code to python notebook to generate list of params per catalogue for every field
per_catalogue_params = [
#     Field       Band        A         B           magnitude limit
    ['ELAIS-N1', 'megacam_r', 2.21267,  -0.0921318, 17],
    ['ELAIS-N1', 'gpc1_r',    1.78775,  -0.0679605, 13],
    ['ELAIS-N1', 'irac1',     2.64814,  -0.110282,  16], # Taking 16 as faint limit
    ['ELAIS-N1', 'master',    3.64842,  -0.183238,  16], # Taking 16 as faint limit
    ['ELAIS-N1', 'suprime_r', 2.57425,  -0.114519,  16],
    ['ELAIS-N1', 'ukidss_k',  3.70217,  -0.183932,  16], # Taking 16 as faint limit
    ['ELAIS-N1', 'wfc_r',     3.61105,  -0.181665,  14]
]   

In [ ]:
for pristine_cat in per_catalogue_params:
    starmask.create_holes('../dmu0/dmu0_GAIA/data/GAIA_' + pristine_cat[0] + '.fits',
                             'dmu4_sm_' + pristine_cat[0] + 
                          '/data/holes_' + pristine_cat[0] + '_' + pristine_cat[1] + '.reg',
                         AB = [pristine_cat[2],pristine_cat[3]],
                         mag_lim = pristine_cat[4])

Again we convert the region files to MOC format.

In [8]:
ORDER=16
for pristine_cat in per_catalogue_params:
    starmask.reg2moc('dmu4_sm_' + pristine_cat[0] + 
                          '/data/holes_' + pristine_cat[0] + '_' + pristine_cat[1] + '.reg',
            '../dmu2/dmu2_field_coverages/' + pristine_cat[0] + '_MOC.fits',
            'dmu4_sm_' + pristine_cat[0] + '/data/holes_' + pristine_cat[0] + '_' + pristine_cat[1] + '_O' + str(ORDER) + '_MOC.fits',
            ra_typ=fields[6][1],
            dec_typ=fields[6][2],
            order=ORDER)


WCSAXES =                    2 / Number of coordinate axes
CRPIX1  =            [astropy.io.fits.card]
     0.5 / Pixel coordinate of reference point
CRPIX2  =                0.5 / Pi [astropy.io.fits.card]
xel coordinate of reference point
CDELT1  =                0.675 / [deg] Coordin [astropy.io.fits.card]
ate increment at reference point
CDELT2  =                0.675 / [deg] Coordina [astropy.io.fits.card]
te increment at reference point
RADECSYSa= 'ICRS    '           / International  [astropy.io.fits.card]
Celestial Ref. System
CUNIT1  = 'deg     '                / Units of coordinate  [astropy.io.fits.card]
increment and value
CUNIT2  = 'deg     '                / Units of coordinate in [astropy.io.fits.card]
crement and value
CTYPE1  = 'RA---TAN'           / 
CTYPE2  = 'DEC--TAN'         [astropy.io.fits.card]
   / 
CRVAL1  =                  247.0 / [deg] Coordinate value at reference poi [astropy.io.fits.card]
nt
CRVAL2  =                  55.1 / [deg] Coordinate value at r